# MediaPipe Face Landmarks

Využijeme knižnicu `mediapipe`a s pomocou detektora face landmarks `face_landmarker_v2_with_blendshapes` určíme a uložíme do textového súbora landmarky pre všetky obrázky zo zložky **faces**.

Landmarky potrebujeme využiť vo FDG (SU2 project).

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 25.8 MB/s eta 0:00:00


In [ ]:
import cv2 as cv
import os
import glob
import gdown
from google.colab.patches import cv2_imshow
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2


In [ ]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

Load img zipped files into folder faces

In [ ]:
ulr='https://drive.google.com/uc?id=1Ms6clnRtQgOjsi0qHq3pEQtItSbUMl8h'
output='faces.zip'
gdown.download(ulr,output,quiet=False)
!unzip "/content/faces.zip" -d "/content"

Downloading...
From: https://drive.google.com/uc?id=1Ms6clnRtQgOjsi0qHq3pEQtItSbUMl8h
To: /content/faces.zip
100%|██████████| 1.02G/1.02G [00:19<00:00, 52.6MB/s]


Výstup streamovania bol skrátený na tento počet posledných riadkov: 5000.
  inflating: /content/faces/1 (1945).jpg  
  inflating: /content/faces/1 (1945).jpg.txt  
  inflating: /content/faces/1 (1946).jpg  
  inflating: /content/faces/1 (1946).jpg.txt  
  inflating: /content/faces/1 (1947).jpg  
  inflating: /content/faces/1 (1947).jpg.txt  
  inflating: /content/faces/1 (195).jpg  
  inflating: /content/faces/1 (195).jpg.txt  
  inflating: /content/faces/1 (1950).jpg  
  inflating: /content/faces/1 (1950).jpg.txt  
  inflating: /content/faces/1 (1951).jpg  
  inflating: /content/faces/1 (1951).jpg.txt  
  inflating: /content/faces/1 (1952).jpg  
  inflating: /content/faces/1 (1952).jpg.txt  
  inflating: /content/faces/1 (1954).jpg  
 extracting: /content/faces/1 (1954).jpg.txt  
  inflating: /content/faces/1 (1955).jpg  
  inflating: /content/faces/1 (1955).jpg.txt  
  inflating: /content/faces/1 (1956).jpg  
  inflating: /content/faces/1 (1956).jpg.txt  
  inflating: /content/faces/

In [ ]:
#@markdown Copied from mediapipe code example https://github.com/googlesamples/mediapipe/blob/main/examples/face_landmarker/python/[MediaPipe_Python_Tasks]_Face_Landmarker.ipynb

#  Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1) #chceme len 1 xicht
detector = vision.FaceLandmarker.create_from_options(options)



In [ ]:
filetypes=["jpg","jpeg","png"]

files=[]
for ft in filetypes:
  files.extend(glob.glob("faces/*."+ft))
files.sort()

for i,f in enumerate(files):
    print("Processing file: {} ".format(f),i,"/",len(files))
    image = mp.Image.create_from_file(f)

    l_file=open(f+"_landm_pipe.txt","w")
    detection_result = detector.detect(image)
    lmks = detection_result.face_landmarks #detekcia
    xyz_coordinates = [[point.x, point.y, point.z] for landmark_list in lmks for point in landmark_list] #extrakcia xyz súradníc, ale sú 0-1

    for u,p in enumerate(xyz_coordinates):
      cord=np.multiply(np.array(p),np.array([image.width,image.height,image.width])).astype(int) #prepočet do súradníc obrázka :)
      l_file.write(str(cord[0])+";"+str(cord[1])+";"+str(cord[0])+"\n")
l_file.close()

Processing file: faces/(11643).jpg  0 / 3456
Processing file: faces/1 (1).jpeg  1 / 3456
Processing file: faces/1 (1).jpg  2 / 3456
Processing file: faces/1 (1).png  3 / 3456
Processing file: faces/1 (10).jpeg  4 / 3456
Processing file: faces/1 (10).jpg  5 / 3456
Processing file: faces/1 (10).png  6 / 3456
Processing file: faces/1 (100).jpg  7 / 3456
Processing file: faces/1 (100).png  8 / 3456
Processing file: faces/1 (1000).jpg  9 / 3456
Processing file: faces/1 (1001).jpg  10 / 3456
Processing file: faces/1 (1002).jpg  11 / 3456
Processing file: faces/1 (1004).jpg  12 / 3456
Processing file: faces/1 (1005).jpg  13 / 3456
Processing file: faces/1 (1006).jpg  14 / 3456
Processing file: faces/1 (1007).jpg  15 / 3456
Processing file: faces/1 (1008).jpg  16 / 3456
Processing file: faces/1 (1009).jpg  17 / 3456
Processing file: faces/1 (101).jpg  18 / 3456
Processing file: faces/1 (101).png  19 / 3456
Processing file: faces/1 (1010).jpg  20 / 3456
Processing file: faces/1 (1011).jpg  21 /

In [ ]:
!zip -r "/content/faces_lmks_pipe.zip" "faces"

Výstup streamovania bol skrátený na tento počet posledných riadkov: 5000.
  adding: faces/1 (4201).jpg.txt (deflated 8%)
  adding: faces/1 (2975).jpg_landm_pipe.txt (deflated 60%)
  adding: faces/1 (6462).jpg (deflated 0%)
  adding: faces/1 (1829).jpg (deflated 1%)
  adding: faces/1 (254).jpg (deflated 3%)
  adding: faces/1 (4268).jpg (deflated 0%)
  adding: faces/1 (3485).jpg.txt (stored 0%)
  adding: faces/1 (3067).jpg.txt (stored 0%)
  adding: faces/1 (3310).jpg_landm_pipe.txt (deflated 59%)
  adding: faces/1 (3179).jpg (deflated 0%)
  adding: faces/1 (1845).jpg.txt (stored 0%)
  adding: faces/1 (2808).jpg (deflated 1%)
  adding: faces/1 (3410).jpg_landm_pipe.txt (deflated 56%)
  adding: faces/1 (215).jpg.txt (deflated 12%)
  adding: faces/1 (1506).jpg (deflated 0%)
  adding: faces/1 (121).jpg.txt (deflated 3%)
  adding: faces/1 (125).jpg (deflated 0%)
  adding: faces/1 (2).jpg_landm_pipe.txt (deflated 56%)
  adding: faces/1 (1332).jpg (deflated 0%)
  adding: faces/1 (3410).jpg.txt 